# Experimento

In [ ]:
#importing libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier #kNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import RFE
# from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [ ]:
casearrest = pd.read_csv('../data/cleaned/casearrest_cl.csv', index_col=0)
jailhistory = pd.read_csv('../data/cleaned/jailhistory_cl.csv', index_col=0)
people = pd.read_csv('../data/cleaned/people_cl.csv', index_col=0)
prisonhistory = pd.read_csv('../data/cleaned/prisonhistory_cl.csv', index_col=0)
charge = pd.read_csv('../data/cleaned/charge_cl.csv', index_col=0)

In [ ]:

def get_best_features(df, target):
    n_of_features = np.arange(1, df.shape[1]+1)
    high_score = 0
    nof = 0
    score_list = []
    for n in n_of_features:
        X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.3, random_state = 0)
        model = LinearRegression()
        rfe = RFE(model, n_features_to_select=n)
        X_train_rfe = rfe.fit_transform(X_train, y_train)
        X_test_rfe = rfe.transform(X_test)
        model.fit(X_train_rfe, y_train)
        score = model.score(X_test_rfe, y_test)
        score_list.append(score)
        if(score>high_score):
            high_score = score
            nof = n
    return nof, high_score, score_list

# Caso `casearrest`

In [ ]:
import seaborn as sns



casearrest_X = casearrest.drop(['decile_score'], axis='columns')
casearrest_y = casearrest['decile_score']
nof, high_score, score_list = get_best_features(casearrest_X, casearrest_y)
print('Casearrest:')
print('N of features:', nof)
print('High score:', high_score)
print('Score list:', score_list)


# plt.figure(figsize=(12,10))
# cor = casearrest_X.corr()
# sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
# plt.show()
# son todas, asi que da igual

In [ ]:
jailhistory_X = jailhistory.drop(['decile_score'], axis='columns')
jailhistory_y = jailhistory['decile_score']
nof, high_score, score_list = get_best_features(jailhistory_X, jailhistory_y)
print('jailhistory:')
print('N of features:', nof)
print('High score:', high_score)
print('Score list:', score_list)

# son todoas, asi que da igual


In [ ]:
people_filter = people[people.columns[0:-9]].drop(columns=['age'])
people_filter.head()
df_race = pd.get_dummies(people_filter['race'], prefix='race', prefix_sep='_')
df_sex = pd.get_dummies(people_filter['sex'], prefix='sex',
                        prefix_sep='_', drop_first=True)

people_filter = pd.concat([people_filter, df_race, df_sex], axis=1).drop('sex', axis=1).drop('race', axis=1)
people_filter.head()
people_X = people_filter.drop(['decile_score'], axis='columns')
people_y = people_filter['decile_score']
nof, high_score, score_list = get_best_features(people_X, people_y)
print('people:')
print('N of features:', nof)
print('High score:', high_score)
print('Score list:', score_list)



In [ ]:
prisonhistory_X = prisonhistory.drop(['decile_score'], axis='columns')
prisonhistory_y = prisonhistory['decile_score']
nof, high_score, score_list = get_best_features(prisonhistory_X, prisonhistory_y)
print('prisonhistory:')
print('N of features:', nof)
print('High score:', high_score)
print('Score list:', score_list)

In [ ]:
charge_X = charge.drop(['decile_score'], axis='columns')
charge_y = charge['decile_score']
nof, high_score, score_list = get_best_features(charge_X, charge_y)
print('charge:')
print('N of features:', nof)
print('High score:', high_score)
print('Score list:', score_list)

In [ ]:
# Parámetros a utilizar en GridSearch
tuned_parameters = {'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                    'weights': ['uniform','distance']}
score = 'f1'

###separamos valores
data_train, data_test, y_train, y_test = train_test_split(people_filter, people_filter['decile_score'], test_size=.30,
                                                    random_state=15, stratify=people_filter['decile_score'])


### dejamos los datos para trabajar
y_test = data_test.decile_score
X_test = data_test.drop(columns=['decile_score'])
y_orig = data_train.decile_score
X_orig = data_train.drop(columns=['decile_score'])


### modelo del original
clf_org = GridSearchCV(KNeighborsClassifier(),
                       param_grid=tuned_parameters,
                       cv=5,
                       scoring="f1_macro")
print(" ORIG Mejor combinación de parámetros:")
clf_org.fit(X_orig,y_orig)
print(clf_org.best_params_)

y_pred = clf_org.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
tuned_parameters = {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                    'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
                    'criterion': ['gini','entropy']}

In [ ]:
clf_org = GridSearchCV(DecisionTreeClassifier(),
                       param_grid=tuned_parameters,
                       cv=5,
                       scoring="f1_macro")
print(" ORIG Mejor combinación de parámetros:")
clf_org.fit(X_orig,y_orig)
print(clf_org.best_params_)

y_pred = clf_org.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
data_train = data_train.reset_index(drop=True)
data_subsampled = data_train[data_train.decile_score == 10]
for i in range(1,10):
    idx = np.random.choice(data_train.loc[data_train['decile_score'] == i].index, size=404, replace=False)
    data_subsampled = pd.concat([data_subsampled,  data_train.iloc[idx]])

In [ ]:
# Parámetros a utilizar en GridSearch
tuned_parameters = {'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                    'weights': ['uniform','distance']}

### dejamos los datos para trabajar
y_test = data_test.decile_score
X_test = data_test.drop(columns=['decile_score'])
y_sub = data_subsampled.decile_score
X_sub = data_subsampled.drop(columns=['decile_score'])


### modelo del original
clf_sub = GridSearchCV(KNeighborsClassifier(),
                       param_grid=tuned_parameters,
                       cv=5,
                       scoring="f1_macro")
print(" SUB Mejor combinación de parámetros:")
clf_sub.fit(X_sub,y_sub)
print(clf_sub.best_params_)

y_pred = clf_sub.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
tuned_parameters = {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                    'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
                    'criterion': ['gini','entropy']}

### dejamos los datos para trabajar
y_test = data_test.decile_score
X_test = data_test.drop(columns=['decile_score'])
y_sub = data_subsampled.decile_score
X_sub = data_subsampled.drop(columns=['decile_score'])


### modelo del original
clf_sub = GridSearchCV(DecisionTreeClassifier(),
                       param_grid=tuned_parameters,
                       cv=5,
                       scoring="f1_macro")
print(" SUB Mejor combinación de parámetros:")
clf_sub.fit(X_sub,y_sub)
print(clf_sub.best_params_)

y_pred = clf_sub.predict(X_test)

print(classification_report(y_test, y_pred))